<a href="https://colab.research.google.com/github/DavidMercadoFaustino/Kaggle/blob/master/Naive_Bayes_FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classificador de noticias
 Esse é um projeto que visa por meio de um dataset com a compilação de noticias de diversos meios de comunicação e sendo categorizado pela veracidade das noticias. 

 Este data set esta no site do Kaggle :https://www.kaggle.com/jruvika/fake-news-detection

In [1]:
# Importação de bibliotecas para utilização na Leitura e na manipulação de dados
import pandas as pd
import numpy as np

### Carregamento do Data Set

In [21]:
# Carregamento pelo pandas por meio do read CSV
df_fake_news = pd.read_csv("/content/Naive_Bayes/data.csv")

In [23]:
df_fake_news.head(5) 'Vizualizar como foi feito o Carregamento'

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [39]:
# Analise da qualidade dos dados importados
df_fake_news.describe()

,URLs,Headline,Body,Label
count,4009,4009,3988,4009
unique,3352,2831,2862,2
top,http://beforeitsnews.com/sports/2017/10/2017-f...,"9/28 Through the 40s: The Gloaming; HBD Bill, ...",a potato battery can light up a room for over ...,Fake News
freq,5,6,143,2137


### Análise Descritiva
A principio seria analisado o corpo do texto(Body) do Data Frame. Como este possivelmente tem dados faltantes dentro da coluna, foi escolhido a manchete(Headline) para se usar como paramentro da analise.

In [11]:
#Importação do Collection para observar melhor os dados que serão analisados
from collections import Counter

In [14]:
Counter(df_fake_news["Label"])

Counter({0: 2137, 1: 1872})

In [24]:
# Alterar os dados da coluna Label para melhor vizualização dos resultados
df_fake_news['Label'].replace(1, 'News',regex=True, inplace=True) 

In [26]:
df_fake_news['Label'].replace(0, 'Fake News',regex=True, inplace=True)

In [27]:
Counter(df_fake_news["Label"])

Counter({'Fake News': 2137, 'News': 1872})

Apesar do numero maior de Fake News foi tomada decisão de continuar sem retirada de elementos do Data Frame, pois a diferença não me pareceu significante para aplicar um drop() em linhas relativas ao Fake News

### Limpeza de Dados
Esta parte é importante quando estamos trabalhando com o Naive Bayes para que ele possa ler os padrões e montar um modelo preditivo.

In [15]:
#importação do NLTK para ajudar no processamento das informações para o Naive Bayes
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# Importação das stopwords para fazer a limpeza da coluna Headline
from nltk.corpus import stopwords

In [17]:
# Estabelecendo as stopwords em inglês
list_stops_words = stopwords.words("english")

In [40]:
df_fake_news["Headline"] = df_fake_news['Headline'].str.lower()

In [41]:
df_fake_news["Headline"].replace('\n',' ', regex=True, inplace=True)

### Separação em Treinamento e Teste
Agora com os dados já tratados temos que estabelecer as porcentagem de treino o nosso modelo de Maachine Learning

In [42]:
#importação do sklearn para criar modelo de previsão
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_fake_news["Headline"], df_fake_news["Label"], 
                                                    test_size=0.30, 
                                                    random_state=42)

Foi estabelecido um treinamento com 70% dos dados e fazer testes com 30% dos dados. 

Esse treinamento é importante para que o algoritimo possa melhorar sua capacidade preditiva.

### Pipeline Model
Foram exportado o **Pipeline** para melhorar o entendimento do codigo, **Naive Bayes** na sua forma de multinomial, ***Count Vectorizer** que segmentas todas as palavras e conta as frequencia de cada uma e o **Tf Idf** Transformer que diminui o peso das palavras mais frequentes.

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [43]:
classify = Pipeline(
                [('vect', CountVectorizer(stop_words= list_stops_words)), 
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB())
                 ])

In [44]:
# treinamento do nosso modelo
classify.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",...
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itse

### Avaliação do Modelo
Agora temos nosso modelo treinado, medimos a capacidade de predição. Que consegue prever cerca de 83%. O Modelo também foi capaz de prever com precisão muito parecida tanto as Fake News quanto as Noticias verdadeira. Mas podemos ver que o modelo tem uma taxa baixa de recall nas Noticias Verdadeiras, que pode ser por conta da diferença amostral das duas variaveis.

In [45]:
# Importação do metrics para medir o aprendizado da maquina
from sklearn import metrics

In [46]:
# Medição da precisão do modelo utilizando as amostras
classify.score(X_test, y_test)

0.8362427265170407

In [47]:
# Capacidade de predição do modelo
preds = classify.predict(X_test)
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

   Fake News       0.83      0.88      0.86       669
        News       0.84      0.78      0.81       534

    accuracy                           0.84      1203
   macro avg       0.84      0.83      0.83      1203
weighted avg       0.84      0.84      0.84      1203



### Predição do Modelo
Agora colocando o modelo para predizer uma Manchete

In [48]:
texto  = "Trump and his allies respond with pseudo-science as US death toll hits 150,000"

In [49]:
# Testando a predição do modelo
classify.predict([texto])

array(['News'], dtype='<U9')

In [51]:
# Base da decisão pela classificação como News
classify.predict_proba([texto])

array([[0.4626499, 0.5373501]])

### Export Models

In [ ]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
filename = 'classification_music.david'
joblib.dump(classify, filename)

['classification_music.david']

In [ ]:
model_david = joblib.load('classification_music.david')

In [ ]:
model_david.predict([texto])

array(['FUNK'], dtype='<U10')